In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score
import matplotlib.pyplot as plt
import numpy as np
from math import log, sqrt
%matplotlib inline 

In [18]:
credit_df = pd.read_csv("/content/drive/MyDrive/DataSets/creditcard.csv")

# Initial Analysis
##### Let's analyze this dataset to understand its characteristics

In [19]:
display(credit_df)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [20]:
#Let's round off values for easy interpretation
credit_df.describe().round(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00,284807.00
mean,94813.86,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,88.35,0.00
std,47488.15,1.96,1.65,1.52,1.42,1.38,1.33,1.24,1.19,1.10,1.09,1.02,1.00,1.00,0.96,0.92,0.88,0.85,0.84,0.81,0.77,0.73,0.73,0.62,0.61,0.52,0.48,0.40,0.33,250.12,0.04
min,0.00,-56.41,-72.72,-48.33,-5.68,-113.74,-26.16,-43.56,-73.22,-13.43,-24.59,-4.80,-18.68,-5.79,-19.21,-4.50,-14.13,-25.16,-9.50,-7.21,-54.50,-34.83,-10.93,-44.81,-2.84,-10.30,-2.60,-22.57,-15.43,0.00,0.00
25%,54201.50,-0.92,-0.60,-0.89,-0.85,-0.69,-0.77,-0.55,-0.21,-0.64,-0.54,-0.76,-0.41,-0.65,-0.43,-0.58,-0.47,-0.48,-0.50,-0.46,-0.21,-0.23,-0.54,-0.16,-0.35,-0.32,-0.33,-0.07,-0.05,5.60,0.00
50%,84692.00,0.02,0.07,0.18,-0.02,-0.05,-0.27,0.04,0.02,-0.05,-0.09,-0.03,0.14,-0.01,0.05,0.05,0.07,-0.07,-0.00,0.00,-0.06,-0.03,0.01,-0.01,0.04,0.02,-0.05,0.00,0.01,22.00,0.00
75%,139320.50,1.32,0.80,1.03,0.74,0.61,0.40,0.57,0.33,0.60,0.45,0.74,0.62,0.66,0.49,0.65,0.52,0.40,0.50,0.46,0.13,0.19,0.53,0.15,0.44,0.35,0.24,0.09,0.08,77.16,0.00
max,172792.00,2.45,22.06,9.38,16.88,34.80,73.30,120.59,20.01,15.59,23.75,12.02,7.85,7.13,10.53,8.88,17.32,9.25,5.04,5.59,39.42,27.20,10.50,22.53,4.58,7.52,3.52,31.61,33.85,25691.16,1.00


# Class Imbalance
### Notice the ratio of fraudulent to non-fraudulent transaction

In [21]:
credit_df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [22]:
credit_df["Class"].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

### Class is biased. 

There are significantly higher number of data points for class label 0 than label 1. This bias will lead to poor quality training for the model

To resolve this , we will use the technique of random undersampling. This will ensure we have equal number of data samples from each label to train the model

In [23]:
xy_array = credit_df.values #store the array values in an array
#start by scaling all the values

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

credit_df['scaled_amount'] = std_scaler.fit_transform(credit_df['Amount'].values.reshape(-1,1))
credit_df['scaled_time'] = std_scaler.fit_transform(credit_df['Time'].values.reshape(-1,1))

credit_df.drop(['Time','Amount'], axis=1, inplace=True)



scaled_amount = credit_df['scaled_amount']
scaled_time = credit_df['scaled_time']

credit_df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
credit_df.insert(0, 'scaled_amount', scaled_amount)
credit_df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

credit_df.head()


,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,0.244964,-1.996583,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.342475,-1.996583,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,1.160686,-1.996562,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,0.140534,-1.996562,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,-0.073403,-1.996541,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


# Random Undersampling
To resolve class imbalance , we will use the technique of random undersampling. This will ensure we have equal number of data samples from each label to train the model


# Steps to do undersampling based modeling

###1) Preserve the original values

###2) Scale all values if necessary

###3) If minority class values are too less, you'll have to do cross validation. Make sure you are also performing stratified sampling to maintain the original ratio of minority to majority classes.
###4) Divide original data into training and testing set. Verify if split is stratified
###5)  Now , sample the datset to ensure samples from all classes are equal in number
###6) Train the model on undersampled data
###7) Test the model on original values

In [24]:
from sklearn.model_selection import train_test_split

#Separate features and labels
X = credit_df.drop('Class', axis=1)
y = credit_df['Class']
original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X,y,stratify=y)

original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.99827251 0.00172749]
[0.99827252 0.00172748]


In [25]:
# Since our classes are highly skewed we should make them equivalent in order to have a normal distribution of the classes.

# Lets shuffle the data before creating the subsamples

credit_df = credit_df.sample(frac=1)

# amount of fraud classes 492 rows.
fraud_credit_df = credit_df.loc[credit_df['Class'] == 1]
non_fraud_credit_df = credit_df.loc[credit_df['Class'] == 0][:492]

normal_distributed_credit_df = pd.concat([fraud_credit_df, non_fraud_credit_df])

# Shuffle dataframe rows
new_credit_df = normal_distributed_credit_df.sample(frac=1, random_state=42)

new_credit_df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
176975,-0.193346,0.593268,-0.770044,-0.228897,1.184393,1.316235,1.135067,0.011264,-0.041304,0.254966,0.286956,-0.420647,-1.570132,0.158560,-0.396511,-0.197953,-0.859864,-0.890856,0.290354,-0.523600,1.211925,0.235709,-0.331053,-1.030288,0.414918,0.517980,-0.751394,-1.031626,0.219622,0.238264,0
252124,-0.350151,1.281336,-1.928613,4.601506,-7.124053,5.716088,1.026579,-3.189073,-2.261897,1.185096,-4.441942,-6.646154,3.827868,-6.518649,0.251137,-12.456706,-0.649166,-1.283145,-2.718560,-0.085466,-2.097385,0.328796,0.602291,-0.541287,-0.354639,-0.701492,-0.030973,0.034070,0.573393,0.294686,1
274493,0.646292,1.499917,1.450032,-2.278838,-1.026091,-1.135426,0.463888,4.781392,-1.929493,1.341419,2.893264,-0.872925,-0.866363,1.003603,-0.157251,-1.075389,-0.757825,-0.491945,0.157810,-0.164447,0.480141,0.373480,0.214166,0.567495,-0.007960,0.828243,-0.430748,0.787980,0.014090,-0.004410,0
154676,-0.344114,0.162739,-1.456876,3.740306,-7.404518,7.440964,-1.549878,-1.661697,-5.757213,1.615011,-2.194881,-6.807135,6.825793,-10.399749,1.836449,-11.872845,-3.642393,-6.044103,-9.750776,-3.116372,0.065317,0.529557,0.957897,0.145339,-0.044704,-0.544962,-0.757757,-0.005352,0.318152,-0.323554,1
153823,-0.344114,0.113905,-19.641857,14.706335,-22.801238,12.114672,-14.898113,0.840778,-28.011293,-11.919639,-8.960922,-18.913243,4.971249,-12.686308,-1.071443,-7.383370,-2.992430,-8.582309,-13.806568,-6.000359,1.422632,3.834649,-8.228874,3.318178,2.585212,0.195009,-1.194803,0.038423,-4.126746,-0.645329,1


In [26]:
from sklearn.model_selection import train_test_split


X = new_credit_df.drop('Class', axis=1)
y = new_credit_df['Class']
# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:

#now we're ready to train.  
clf = LogisticRegressionCV(max_iter=10000,cv=5, penalty='elasticnet',  l1_ratios=[0.1, 0.5, 1.0],
                           solver='saga',
                           random_state=0).fit(X_train, y_train)

In [28]:
print('accuracy on training data = ', clf.score(X_train, y_train))


print('accuracy on test data = ', clf.score(original_Xtest, original_ytest))
      

accuracy on training data =  0.9440914866581956
accuracy on test data =  0.9788067750905873


##  
##  Confusion matrix

The definition a confusion matrix is such that Cij is equal to the number of observations known to be in group i and predicted to be in group j.

That means that entries in the confusion matrix C00, C10, C01, C11 represent the following values:
- C00 = true negatives
- C10 = false negatives
- C11 = true positives
- C01 = false positives


# Experiment with threshold values to get best TP and FP numbers


In [29]:
#Generally metrics require probability predictions.  
probs_te = clf.predict_proba(original_Xtest)
prob1 = [x[1] for x in probs_te] #linear model returns prob of both binary outcomes

#threshold at 0.5
y_te_05 = [int(x + 0.5) for x in prob1]
y_te_75 = [int(x + 0.25) for x in prob1]
y_te_25 = [int(x + 0.75) for x in prob1]

conf_mat05 = confusion_matrix(original_ytest, y_te_05)
conf_mat75 = confusion_matrix(original_ytest, y_te_75)
conf_mat25 = confusion_matrix(original_ytest, y_te_25)

print('confusion matrix with 0.5 threshold \n', conf_mat05)
print('confusion matrix with 0.75 threshold \n', conf_mat75)
print('confusion matrix with 0.25 threshold \n', conf_mat25)

confusion matrix with 0.5 threshold 
 [[69580  1499]
 [   10   113]]
confusion matrix with 0.75 threshold 
 [[70344   735]
 [   14   109]]
confusion matrix with 0.25 threshold 
 [[65066  6013]
 [    6   117]]


##  
##  Feature Importance

For penalized linear regression, we'll use the coefficient magnitudes to indicate the value of the corresponding features.  Those are available as an output from the training process.  In order of importance here are the most important features (remember that it's that absolute value of the coefficient that counts).

1. ca       -0.789812
2. cp        0.708058
3. thalach   0.581009
4. old peak -0.575971
5. thal     -0.462587


In [30]:
coef = clf.coef_
coef=np.array([abs(xi) for xi in coef])
columns = new_credit_df.columns[0:30]
plm_importance_df = pd.DataFrame.from_dict(dict(zip(columns, coef)), orient='index', columns=columns)
display(plm_importance_df.T.sort_values(by='scaled_amount',ascending=False).head(20))

,scaled_amount
V4,0.632758
V14,0.575927
V12,0.279500
V10,0.267236
V8,0.229798
V23,0.194319
V3,0.189483
scaled_amount,0.186835
V11,0.155163
V19,0.128150
